In [ ]:
!pip install langchain openai crewai

In [ ]:
import json

sample_data = {
    "lead": {
        "name": "Jane Doe",
        "job_title": "VP, Sales",
        "company": "TechNova",
        "interests": ["B2B sales automation", "customer engagement", "AI solutions"]
    },
    "product": {
        "name": "SalesPilot AI",
        "value_props": [
            "automates cold email outreach",
            "personalizes messages using AI",
            "integrates seamlessly with CRM"
        ],
        "website": "https://salespilot.example.com"
    }
}

# Save to JSON file
with open("lead_and_product.json", "w") as f:
    json.dump(sample_data, f, indent=2)

In [ ]:
# data_handler.py (can be made its own .py file if running locally)
import json

def load_data(json_path="lead_and_product.json"):
    with open(json_path, "r") as f:
        return json.load(f)

data = load_data()
print(data)

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # Paste your key here for Colab testing.

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo") # or "gpt-4" if available

In [ ]:
def email_generation_prompt(lead, product):
    return f"""
You are an expert SDR. Given the following lead and product info, generate:
1. A compelling email subject (brief).
2. A personalized email body (no more than 120 words, conversational, referencing the lead's role/interests and the product's value).

Lead:
- Name: {lead["name"]}
- Job Title: {lead["job_title"]} at {lead["company"]}
- Interests: {', '.join(lead['interests'])}

Product:
- Name: {product["name"]}
- Value Propositions: {', '.join(product['value_props'])}
- Website: {product['website']}

Return JSON:
{{ "subject": "...", "body": "..." }}
    """

def generate_email(llm, lead, product):
    prompt = email_generation_prompt(lead, product)
    resp = llm.invoke(prompt)
    try:
        # Extract JSON from LLM output (if proper JSON, else fallback)
        import re
        import json
        match = re.search(r"\{.*\}", resp.content, re.DOTALL)
        if match:
            return json.loads(match.group())
        else:
            return {"subject":"Parsing failed", "body": resp.content}
    except Exception as e:
        return {"subject": "Error", "body": str(e)}

In [ ]:
from crewai import Agent, Task, Crew

# Agent that does the reasoning and triggers LLM via the modular function
class EmailReasoningAgent(Agent):
    def run(self, data, llm):
        lead = data["lead"]
        product = data["product"]
        return generate_email(llm, lead, product)

# Define the agent
email_agent = EmailReasoningAgent(name="Email Reasoning Agent")

# Define a task for the agent to execute
def email_task(agent, data, llm):
    return agent.run(data, llm)

# Orchestrate with CrewAI "Crew"
crew = Crew(agents=[email_agent])

In [ ]:
# Load data
data = load_data("lead_and_product.json")

# Run the agent and get the email content
result = email_task(email_agent, data, llm)

print("=== Email Suggestion ===")
print("Subject:", result['subject'])
print("\nBody:\n", result['body'])

In [ ]:
# Dockerfile

FROM python:3.11

WORKDIR /app

# Install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy source files
COPY . .

CMD ["python", "main.py"]

In [ ]:
langchain
openai
crewai